In [30]:
import torch
import numpy as np
import os
!pip install transformers

In [8]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/MyDrive/Colab Notebooks/Legal Search Tool/data/text'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# Using the Bert Classifier from Hugging face

from transformers import BertTokenizer

# Downloading the base bert classifier which treats upper and lower case as same
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [76]:
# Reading the documents into a list
documents = []
file_list = []
for files in os.listdir(dir):
  try:
    if files.split('.')[1]=='txt':
      file_list.append(files)
      with open ( dir + '/'+ files, 'r') as file:
        documents.append(file.readlines()[0])
  except:
    print(files)

In [77]:
# Create input_ids and attention masks for the documents
tokenized = []
for document in documents:
  tokenized.append ( bert_tokenizer ( document, max_length = 128, padding = 'max_length', return_tensors = 'pt'))

In [78]:
# Importing a bert model to encode the data 
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [79]:
# Encoding the sentences
from tqdm import tnrange
model_output = []
for token in tnrange(len(tokenized), desc = 'Encoding the sentences'):
  model_output.append( bert_model ( tokenized[token]['input_ids'], attention_mask = tokenized[token]['attention_mask'] ) )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


Encoding the sentences:   0%|          | 0/50 [00:00<?, ?it/s]

In [80]:
np.save(dir+'/bert_transformed.npy',model_output)

In [81]:
search_text = "Golap Sundari and two sons Rajendra and Jogendra perform the seva of the diety and after their death"

In [82]:
def pre_process(text):
  pre_process_text = bert_tokenizer ( text, max_length = 128, padding = 'max_length', return_tensors = 'pt')
  vectorized = bert_model ( pre_process_text['input_ids'], attention_mask = pre_process_text['attention_mask'] ) 
  return vectorized

In [83]:
search_vector = pre_process(search_text).pooler_output

In [84]:
from scipy import spatial
document_idx = None
min = 1000
for i in tnrange(len(model_output)):
  cosine = spatial.distance.cosine(search_vector.detach().numpy(), model_output[i].pooler_output.detach().numpy())
  if cosine < min :
    min = cosine
    document_idx = i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


  0%|          | 0/50 [00:00<?, ?it/s]

In [85]:
file_list[document_idx]

'2009_B_16.txt'